In [35]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
gbm = lgb.LGBMRanker()
from sklearn.model_selection import train_test_split

In [36]:
# Specify the file name
file_name = 'df_OptionB.csv'

# Specify the full file path
file_path = r'C:\Users\jeroe\OneDrive\Documenten\3. VU\Master Econometrics\Data Mining Techniques\Assignment 2\\' + file_name

# Read the dataset using pandas
df = pd.read_csv(file_path)

In [37]:
df.columns

Index(['srch_id', 'prop_id', 'prop_starrating', 'prop_review_score',
       'prop_brand_bool', 'prop_location_score1', 'prop_location_score2',
       'prop_log_historical_price', 'price_usd', 'promotion_flag',
       'srch_length_of_stay', 'srch_booking_window', 'srch_adults_count',
       'srch_room_count', 'srch_saturday_night_bool',
       'orig_destination_distance', 'click_bool', 'booking_bool', 'score',
       'avg_comp_rate', 'avg_comp_inv', 'abroad_bool', 'children_bool',
       'srch_query_affinity_score_low', 'srch_query_affinity_score_high',
       'starrating_diff_low', 'starrating_diff_high', 'usd_diff_low',
       'usd_diff_high'],
      dtype='object')

In [38]:
click_indices = df[df['click_bool'] == 1].index
random_indices = np.random.choice(click_indices, len(df.loc[df['click_bool'] == 1]), replace=False)
click_sample = df.loc[random_indices]

not_click = df[df['click_bool'] == 0].index
random_indices = np.random.choice(not_click, sum(df['click_bool']), replace=False)
not_click_sample = df.loc[random_indices]

df = pd.concat([not_click_sample, click_sample], axis=0)

print("Non-click impressions: ", len(df[df['click_bool'] == 0]))
print("Click impression: ", len(df[df['click_bool'] == 1]))
print("Total number of records in resampled data: ", len(df))

Non-click impressions:  221879
Click impression:  221879
Total number of records in resampled data:  443758


In [39]:
df = df.sort_values(by='srch_id')

In [40]:
group = list(df['srch_id'].value_counts())

In [41]:
y=df['score']
X=df.drop(['score'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1)

In [42]:
group_train = list(X_train['srch_id'].value_counts())
group_test = list(X_test['srch_id'].value_counts())

In [43]:
X_train = X_train.drop(['click_bool', 'booking_bool', 'srch_id', 'prop_id'], axis=1)
X_test = X_test.drop(['click_bool', 'booking_bool', 'srch_id', 'prop_id'], axis=1)

In [45]:
gbm.fit(X_train, y_train, group=group_train,
        eval_set=[(X_train, y_train), (X_test, y_test)], eval_group=[group_train, group_test],
        eval_at=5, early_stopping_rounds=50)

c:\Users\jeroe\anaconda3\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[1]	training's ndcg@5: 0.948642	valid_1's ndcg@5: 0.984129
[2]	training's ndcg@5: 0.949601	valid_1's ndcg@5: 0.984327
[3]	training's ndcg@5: 0.950211	valid_1's ndcg@5: 0.984357
[4]	training's ndcg@5: 0.950479	valid_1's ndcg@5: 0.984386
[5]	training's ndcg@5: 0.950738	valid_1's ndcg@5: 0.98444
[6]	training's ndcg@5: 0.950928	valid_1's ndcg@5: 0.984432
[7]	training's ndcg@5: 0.951178	valid_1's ndcg@5: 0.984435
[8]	training's ndcg@5: 0.95136	valid_1's ndcg@5: 0.984421
[9]	training's ndcg@5: 0.951615	valid_1's ndcg@5: 0.984507
[10]	training's ndcg@5: 0.951696	valid_1's ndcg@5: 0.984461
[11]	training's ndcg@5: 0.951762	valid_1's ndcg@5: 0.984515
[12]	training's ndcg@5: 0.952003	valid_1's ndcg@5: 0.984607
[13]	training's ndcg@5: 0.952106	valid_1's ndcg@5: 0.984691
[14]	training's ndcg@5: 0.9523	valid_1's ndcg@5: 0.984719
[15]	training's ndcg@5: 0.952435	valid_1's ndcg@5: 0.984719
[16]	training's ndcg@5: 0.952595	valid_1's ndcg@5: 0.984775
[17]	training's ndcg@5: 0.952784	valid_1's ndcg@5: 0.

LGBMRanker()

In [46]:
test_pred = gbm.predict(X_test)

In [47]:
X_test["predicted_ranking"] = test_pred
X_test.sort_values("predicted_ranking", ascending=False)

,prop_starrating,prop_review_score,prop_brand_bool,prop_location_score1,prop_location_score2,prop_log_historical_price,price_usd,promotion_flag,srch_length_of_stay,srch_booking_window,...,avg_comp_inv,abroad_bool,children_bool,srch_query_affinity_score_low,srch_query_affinity_score_high,starrating_diff_low,starrating_diff_high,usd_diff_low,usd_diff_high,predicted_ranking
4106659,2,4.0,1,1.95,0.7200,4.48,55.00,0,1,0,...,0.0,0,0,1,0,1,0,1,0,2.170379
1838407,3,4.5,1,0.00,0.6923,5.15,114.52,0,1,5,...,0.0,0,0,0,0,1,0,1,0,2.067948
4589358,4,4.0,1,1.79,0.9195,4.68,68.24,0,1,33,...,0.0,0,0,0,0,0,0,0,0,2.039637
1380219,0,4.5,1,1.10,0.8885,3.96,59.00,0,1,0,...,1.0,0,0,0,0,0,0,0,0,2.003859
2342760,2,4.5,1,1.10,0.7025,4.60,75.00,0,1,3,...,0.0,0,0,0,0,1,0,1,0,1.997004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3480152,2,1.5,1,0.00,0.0046,4.58,50.00,0,1,6,...,0.0,0,0,0,0,0,1,0,1,-1.694958
2086813,0,5.0,0,0.00,0.0013,4.73,195.95,0,2,8,...,0.0,0,0,0,0,0,0,0,0,-1.715776
4926707,2,5.0,1,1.10,0.0004,4.44,177.00,0,1,51,...,0.0,0,0,0,0,0,0,0,0,-1.876642
3553668,0,5.0,0,1.79,0.0012,4.46,280.00,0,1,15,...,0.0,0,1,0,0,0,0,0,0,-1.908251
